In [3]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [7]:
"""
Test the vina docking and extracting score.
"""

import sys
import os


from datetime import datetime

from REVIVAL.zs.vina import dock_lib
from REVIVAL.util import checkNgen_folder

if __name__ == "__main__":

    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    # log outputs
    f = open(
        os.path.join(
            checkNgen_folder("logs/zs/vina/dock"),
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.out",
        ),
        "w",
    )
    sys.stdout = f

    for chai_dir in [ 
        "zs/chai/mut_structure/PfTrpB-4bromo-plp", 
        "zs/chai/mut_structure/PfTrpB-4cyano-plp",
        "zs/chai/mut_structure/PfTrpB-5chloro-plp",
        "zs/chai/mut_structure/PfTrpB-6chloro-plp",
        "zs/chai/mut_structure/PfTrpB-7bromo-plp",
        "zs/chai/mut_structure/PfTrpB-7indo-plp",
        "zs/chai/mut_structure/PfTrpB-7methyl-plp",
        "zs/chai/mut_structure/PfTrpB-56chloro-plp",
        "zs/chai/mut_structure/PfTrpB-5cyano-plp",
    ]:
        dock_lib(chai_dir, "inactivated-cofactor")


    # def dock_lib(
    #     chai_dir: str,
    #     cofactor_type: str,
    #     vina_dir: str = "vina",
    #     pH: float = 7.4,
    #     method='vina',
    #     size_x=15.0,
    #         size_y=15.0, 
    #         size_z=15.0,
    #         num_modes=9, # Dunno check vina docks using the defaut
    #         exhaustiveness=32 
    # ):

    f.close()

1 molecule converted
[01:40:26] Running Uncharger
[01:40:27] Running Uncharger
[01:40:27] Removed negative charge.
[01:40:27] Removed negative charge.
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
1 molecule converted
1 molecule converted
[01:40:45] Running Uncharger
[01:40:46] Running Uncharger
[01:40:46] Removed negative charge.
[01:40:46] Removed negative charge.
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
1 molecule converted
1 molecule converted
[01:41:02] Running Uncharger
[01:41:03] Running Uncharger
[01:41:03] Removed negative charge.
[01:41:03] Removed negative charge.
*** Open Babel Warning  in parseAtomRecord
  Problems readi

KeyError: 'CA'

In [17]:
def pdb_to_pdbqt_protein(input_path: str, output_path=None, pH: float = 7.4):
    """
    Convert a pdb file to a pdbqt file.
    """
    # Need to first remove stuff that is sometimes added by
    lines = []
    with open(input_path, "r+") as fin:
        for line in fin:
            if (
                line.split(" ")[0] not in ["ENDBRANCH", "BRANCH", "ROOT", "ENDROOT"]
                and "Fe" not in line
            ):  # Add in the removal of the Iron bit
                lines.append(line)
    with open(input_path, "w+") as fout:
        for line in lines:
            fout.write(line)

    output_path = output_path if output_path else input_path.replace(".pdb", ".pdbqt")
    # os.system(
    #     f"obabel {input_path} -xr -p {pH} --partialcharge gasteiger -O {output_path}"
    # )
    os.system(
        f"obabel {input_path} -xr --partialcharge gasteiger -O {output_path}"
    )
    # Now we also want to be cheeky and remove any secondary model parts from the file
    # This is a hacky way to keep a bound heme or something, seems to work fine.
    lines = []
    with open(output_path, "r+") as fin:
        for line in fin:
            if line.split(" ")[0] not in ["MODEL", "TER", "ENDMDL", "REMARK"]:
                lines.append(line)
    with open(output_path, "w+") as fout:
        for line in lines:
            if "ENDMDL" not in line:
                fout.write(line)
        fout.write("TER\n")


In [18]:
pdb_to_pdbqt_protein(
        input_path="/disk2/fli/REVIVAL2/zs/vina/PfTrpB-4bromo-plp/I165C:I183S:Y301A_4/I165C:I183S:Y301A_4/I165C:I183S:Y301A_4.pdb", output_path="/disk2/fli/REVIVAL2/zs/vina/PfTrpB-4bromo-plp/I165C:I183S:Y301A_4/I165C:I183S:Y301A_4/I165C:I183S:Y301A_4.pdbqt", pH=7.4
    )

1 molecule converted


In [1]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.249.233 49752 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8085', 'VSCODE_CLI_REQUIRE_TOKEN': '0bd3ec2e-f17e-4d6b-888d-5ed25764ec79', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-fabdb6a30b49f79a7aba0f2ad9df9b399473380f/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGEN